In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Load and preprocess the data
data = pd.read_csv('weather_data.csv')
data = (data - data.min()) / (data.max() - data.min())
data = torch.tensor(data.values, dtype=torch.float32)

# Split the data into training and testing sets
train_data = data[:25]
test_data = data[25:]

# Prepare the training data
seq_length = 5
train_sequences = []
train_labels = []
for i in range(len(train_data) - seq_length):
    train_sequences.append(train_data[i:i+seq_length])
    train_labels.append(train_data[i+seq_length, -1])  # Use the last column as the target
train_sequences = torch.stack(train_sequences)
train_labels = torch.stack(train_labels)

# Prepare the testing data
test_sequences = []
test_labels = []
for i in range(len(test_data) - seq_length):
    test_sequences.append(test_data[i:i+seq_length])
    test_labels.append(test_data[i+seq_length, -1])  # Use the last column as the target
test_sequences = torch.stack(test_sequences)
test_labels = torch.stack(test_labels)

# Define the model hyperparameters
input_size = train_sequences.shape[2]
hidden_size = 32
num_layers = 2
output_size = 1  # Since we are predicting a single column

num_epochs = 100
learning_rate = 0.01

# Initialize the LSTM model
model = LSTM(input_size, hidden_size, num_layers, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(train_sequences)
    loss = criterion(outputs.squeeze(), train_labels)  # Squeeze the output tensor to match the shape of train_labels
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Switch the model to evaluation mode
model.eval()

# Prepare the testing data
test_sequences = []
test_labels = []
for i in range(len(test_data) - seq_length):
    test_sequences.append(test_data[i:i+seq_length])
    test_labels.append(test_data[i+seq_length])
test_sequences = torch.stack(test_sequences)
test_labels = torch.stack(test_labels)

# Make predictions on the test data
with torch.no_grad():
    test_predictions = model(test_sequences)

# Denormalize the data
denormalized_predictions = test_predictions * (data[:, -1].max() - data[:, -1].min()) + data[:, -1].min()
denormalized_actual = test_data[:, -1] * (data[:, -1].max() - data[:, -1].min()) + data[:, -1].min()

# Plot the predictions (orange line) and the actual values (blue line)
plt.plot(denormalized_actual, color='blue', label='Actual')
plt.plot(denormalized_predictions, color='orange', label='Predicted')

# Set x-axis label and title
plt.xlabel('Time')
plt.title('Predictions vs Actual')

# Display legend
plt.legend()

# Show the plot
plt.show()




/home/rpgsbs/r03mb21/anaconda3/envs/Matt/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2090])) that is different to the input size (torch.Size([2090, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan


RuntimeError: Expected sequence length to be larger than 0 in RNN